In [1]:
! pip install statsmodels


In [2]:
#Siyuan Li
#last update: 2024/2/1


from sklearn.linear_model import LassoCV, ElasticNetCV,LogisticRegression
from sklearn.model_selection import cross_val_predict,train_test_split,KFold
from sklearn.metrics import roc_auc_score, roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor


import pandas as pd
import os

In [3]:
result_df = pd.read_csv("/data/Clockbase/Methylation/sample_disease.txt",sep=" ", header=0)  

#filter samples with disease label
#selected_columns = df_sample.columns[df_sample.iloc[0].notna()]
#result_df = pd.concat([result_df, df_sample[selected_columns]], axis=1)

result_df.head()


/tmp/ipykernel_52122/3790493289.py:1: DtypeWarning: Columns (4,5,7,12,25,26,30,33,35,39,40,49,51,55,74,86,94,100) have mixed types. Specify dtype option on import or set low_memory=False.
  result_df = pd.read_csv("/data/Clockbase/Methylation/sample_disease.txt",sep=" ", header=0)


,sample_id,age,age_at_menarche,age_group,antibiotic_use_during_pregnancy,arousal_score,asthma_medication,atopy,attention_score,birth_mode,...,treatment,twin,urinary_total_arsenic,vaginal_infection,viral_load,weight,years_of_smoking,sample_type,platform,urinary_tract_infection
GSM1068826,GSM1068826,88.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,disease tissue,450K,NaN
GSM1068827,GSM1068827,92.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,disease tissue,450K,NaN
GSM1068829,GSM1068829,93.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,disease tissue,450K,NaN
GSM1068832,GSM1068832,96.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,disease tissue,450K,NaN
GSM1068833,GSM1068833,86.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,disease tissue,450K,NaN


In [4]:
print(result_df['disease'].unique())
print(result_df['platform'].unique())

["Alzheimer's disease" nan 'asthma' 'autism spectrum disorder'
 'childhood asthma' "Crohn's disease" "crohn's disease" 'Down syndrome'
 "Graves' disease" "Huntington's disease"
 'intellectual disability and congenital anomalies' 'Kabuki syndrome'
 'multiple sclerosis' 'nephrogenic rest' 'panic disorder'
 "Parkinson's disease" 'preeclampsia' 'psoriasis' 'respiratory allergy'
 'rheumatoid arthritis' 'schizophrenia' 'Silver Russell syndrome'
 "Sjogren's syndrome" 'spina bifida' 'stroke'
 'systemic insulin resistance' 'systemic lupus erythematosus'
 'systemic sclerosis' 'type 2 diabetes' 'Ulcerative colitis'
 'ulcerative colitis']
['450K']


In [5]:
num_rows = result_df.shape[0]
print("数据框有", num_rows, "行。")


数据框有 14501 行。


In [6]:
nan_rows = result_df[result_df['disease'].isna()]
num_nan_rows = nan_rows.shape[0]
print(" 'disease' 列中有", num_nan_rows, "行的值是 NaN。")


 'disease' 列中有 9188 行的值是 NaN。


In [7]:
'''
result_df = result_df.dropna(subset=['disease'])
print(result_df['disease'].unique())
num_rows = result_df.shape[0]
print("数据框有", num_rows, "行。")
'''

'\nresult_df = result_df.dropna(subset=[\'disease\'])\nprint(result_df[\'disease\'].unique())\nnum_rows = result_df.shape[0]\nprint("数据框有", num_rows, "行。")\n'

In [8]:
result_df.head()

,sample_id,age,age_at_menarche,age_group,antibiotic_use_during_pregnancy,arousal_score,asthma_medication,atopy,attention_score,birth_mode,...,treatment,twin,urinary_total_arsenic,vaginal_infection,viral_load,weight,years_of_smoking,sample_type,platform,urinary_tract_infection
GSM1068826,GSM1068826,88.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,disease tissue,450K,NaN
GSM1068827,GSM1068827,92.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,disease tissue,450K,NaN
GSM1068829,GSM1068829,93.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,disease tissue,450K,NaN
GSM1068832,GSM1068832,96.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,disease tissue,450K,NaN
GSM1068833,GSM1068833,86.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,disease tissue,450K,NaN


In [9]:
#df2 = pd.read_csv("/data/Clockbase/Methylation/disease/GSE43414.csv.gz",compression='gzip')  
#df2.head()


In [10]:
#df2t=df2.transpose()
#df2t.head()

In [11]:
#print(df2t.index)

In [12]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

In [13]:
unique_disease = result_df['disease'].unique()
print(unique_disease)

["Alzheimer's disease" nan 'asthma' 'autism spectrum disorder'
 'childhood asthma' "Crohn's disease" "crohn's disease" 'Down syndrome'
 "Graves' disease" "Huntington's disease"
 'intellectual disability and congenital anomalies' 'Kabuki syndrome'
 'multiple sclerosis' 'nephrogenic rest' 'panic disorder'
 "Parkinson's disease" 'preeclampsia' 'psoriasis' 'respiratory allergy'
 'rheumatoid arthritis' 'schizophrenia' 'Silver Russell syndrome'
 "Sjogren's syndrome" 'spina bifida' 'stroke'
 'systemic insulin resistance' 'systemic lupus erythematosus'
 'systemic sclerosis' 'type 2 diabetes' 'Ulcerative colitis'
 'ulcerative colitis']


In [14]:
# find path
folder_path = "/data/Clockbase/Methylation/disease/"
file_names = os.listdir(folder_path)
# output_path
output_path="/data/Clockbase/Methylation/disease_predictor/"

unique_disease = result_df['disease'].unique()
disease_gse_dict = {}
train_ratio = 0.7
# 遍历每个不同的 'project_id'
for disease in unique_disease:
    
    if pd.isna(disease):
        continue
    
    selected_rows1 = result_df.loc[result_df['disease'] == disease]
    unique_project_ids = selected_rows1['project_id'].unique()
    print(f"在 'disease' 是 '{disease}' 的行中，'roject_id' 列有 {len(unique_project_ids)} 种不同的值：")
    print(unique_project_ids)
    
    disease_gse_dict[disease] = list(unique_project_ids)
display(disease_gse_dict)


在 'disease' 是 'Alzheimer's disease' 的行中，'roject_id' 列有 7 种不同的值：
['GSE43414' 'GSE59685' 'GSE72778' 'GSE66351' 'GSE80970' 'GSE79144'
 'GSE53740']
在 'disease' 是 'asthma' 的行中，'roject_id' 列有 4 种不同的值：
['GSE85566' 'GSE104471' 'GSE52074' 'GSE56553']
在 'disease' 是 'autism spectrum disorder' 的行中，'roject_id' 列有 1 种不同的值：
['GSE50759']
在 'disease' 是 'childhood asthma' 的行中，'roject_id' 列有 4 种不同的值：
['GSE65163' 'GSE104087' 'GSE109446' 'GSE40576']
在 'disease' 是 'Crohn's disease' 的行中，'roject_id' 列有 2 种不同的值：
['GSE87640' 'GSE87648']
在 'disease' 是 'crohn's disease' 的行中，'roject_id' 列有 1 种不同的值：
['GSE32148']
在 'disease' 是 'Down syndrome' 的行中，'roject_id' 列有 4 种不同的值：
['GSE63347' 'GSE73747' 'GSE52588' 'GSE64495']
在 'disease' 是 'Graves' disease' 的行中，'roject_id' 列有 1 种不同的值：
['GSE71955']
在 'disease' 是 'Huntington's disease' 的行中，'roject_id' 列有 1 种不同的值：
['GSE72778']
在 'disease' 是 'intellectual disability and congenital anomalies' 的行中，'roject_id' 列有 1 种不同的值：
['GSE89353']
在 'disease' 是 'Kabuki syndrome' 的行中，'roject_id' 列

{"Alzheimer's disease": ['GSE43414',
  'GSE59685',
  'GSE72778',
  'GSE66351',
  'GSE80970',
  'GSE79144',
  'GSE53740'],
 'asthma': ['GSE85566', 'GSE104471', 'GSE52074', 'GSE56553'],
 'autism spectrum disorder': ['GSE50759'],
 'childhood asthma': ['GSE65163', 'GSE104087', 'GSE109446', 'GSE40576'],
 "Crohn's disease": ['GSE87640', 'GSE87648'],
 "crohn's disease": ['GSE32148'],
 'Down syndrome': ['GSE63347', 'GSE73747', 'GSE52588', 'GSE64495'],
 "Graves' disease": ['GSE71955'],
 "Huntington's disease": ['GSE72778'],
 'intellectual disability and congenital anomalies': ['GSE89353'],
 'Kabuki syndrome': ['GSE97362', 'GSE116300'],
 'multiple sclerosis': ['GSE40360', 'GSE106648', 'GSE88824', 'GSE43976'],
 'nephrogenic rest': ['GSE59157'],
 'panic disorder': ['GSE102468'],
 "Parkinson's disease": ['GSE57360',
  'GSE111223',
  'GSE111629',
  'GSE72774',
  'GSE72776'],
 'preeclampsia': ['GSE44667',
  'GSE100197',
  'GSE103253',
  'GSE57767',
  'GSE73375',
  'GSE75196'],
 'psoriasis': ['GSE6331

In [16]:
def preprocess_data(data):
    # Extracting CpG columns
    cpg_columns = data.columns[:-2]

    # Excluding outliers based on visual inspection
    outlier_threshold1 = 10
    data = data[data[cpg_columns].apply(lambda x: (x <= outlier_threshold1).all(), axis=1)]

    # Sample Exclusions: Samples excluded if ≥1% of CpGs had a detection p-value exceeding 0.05
    outlier_threshold2 = data[cpg_columns].mean() + 1.96 * data[cpg_columns].std() / np.sqrt(len(data))
    columns_to_retain = []
    for cpg_column in cpg_columns:
        num_outliers = (data[cpg_column] > outlier_threshold2[cpg_column]).sum()
        if num_outliers <= 0.01 * len(data):
            columns_to_retain.append(cpg_column)
    data = data[columns_to_retain]

    # Remove CpGs with missing values
    df_cleaned = data.dropna(axis=1)

    # Normalization
    scaler = StandardScaler()
    df_cleaned = scaler.fit_transform(df_cleaned.iloc[:, :-2])

    return df_cleaned

In [19]:
MODEL_RESULT = pd.DataFrame(columns=['Disease', 'Best_lambda_index','Best_model_coefs', 'AUC'])
for disease in unique_disease:
  
  
    gse_list = disease_gse_dict[disease]
      
    if len(gse_list) == 1:
        train_set = gse_list
        test_set = gse_list #Set it same for now
    else:
        # Calculate the number of training and testing sets
        num_train = max(1, int(len(gse_list) * train_ratio))
        num_test = len(gse_list) - num_train
        
        # Randomly select training and testing sets from all GSE files of the disease
        np.random.shuffle(gse_list)
        train_set = gse_list[:num_train]
        test_set = gse_list[num_train:]



    train_data = pd.DataFrame()
    # Load training set data
    for gse_file in train_set:
        file_path = folder_path + gse_file + ".csv.gz"  
        dt = pd.read_csv(file_path,compression='gzip')
        dt_trans=dt.transpose()
        train_data=pd.concat([train_data, dt_trans], axis=0)
    display(train_data.index)
    train_data_matched = train_data.merge(result_df[['disease', 'project_id']], left_index=True, right_index=True, how='left')

    test_data = pd.DataFrame()
    # Load testing set data
    for gse_file in test_set:
        file_path = folder_path + gse_file + ".csv.gz"  
        dt = pd.read_csv(file_path,compression='gzip')
        dt_trans=dt.transpose()
        test_data=pd.concat([test_data, dt_trans], axis=0)
    test_data_matched = test_data.merge(result_df[['disease', 'project_id']], left_index=True, right_index=True, how='left')
    
    # Preprocess train_data_matched
    X_train = preprocess_data(train_data_matched)
    y_train = train_data_matched.iloc[:, -2]

    # Preprocess test_data_matched
    test_data_matched_processed = preprocess_data(test_data_matched)
    X_test = preprocess_data(test_data_matched)
    y_test = test_data_matched.iloc[:, -2]
    
    
    
    model = LassoCV(alphas=[1], cv=10, random_state=42)
    model.fit(X_train, y_train)

    best_lambda_index = np.argmin(model.lambda_path_['val_mean_score'])
    best_model_coefs = model.coef_[:, best_lambda_index]

    predictions = model.predict(X_test, lamb=model.lambda_path_['lambda_max'])

  
    auc = roc_auc_score(y_test, predictions )
   
    '''
    fpr, tpr, _ = roc_curve(y_test, predictions)
    roc_auc = auc(fpr, tpr)
    print(f"ROC AUC: {roc_auc}")
    '''
    
    MODEL_RESULT = MODEL_RESULT.append({'Disease': disease, 'Best_lambda_index':best_lambda_index, 'Best_model_coefs': best_model_coefs, 'AUC': auc}, ignore_index=True)


disease_name = disease.replace(" ", "_") + ".csv"
MODEL_RESULT.to_csv(os.path.join(output_path, 'MODEL_RESULT_'+disease_name), index=False)

Index(['sample_id', 'GSM1068826', 'GSM1068827', 'GSM1068829', 'GSM1068832',
       'GSM1068833', 'GSM1068834', 'GSM1068835', 'GSM1068838', 'GSM1068839',
       ...
       'GSM2086838', 'GSM2086839', 'GSM2086843', 'GSM2086846', 'GSM2086847',
       'GSM2086849', 'GSM2086851', 'GSM2086852', 'GSM2086855', 'GSM2086857'],
      dtype='object', length=816)

: 